In [65]:
import os
PROJECT = "tatvic-gcp-dev-team" # REPLACE WITH YOUR PROJECT ID
BUCKET = 'data-etl-process1' # REPLACE WITH YOUR BUCKET NAME
REGION = 'asia-east1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1



In [138]:
# For Python Code
# Model Info
MODEL_NAME = 'virality'
# Model Version
MODEL_VERSION = 'v1'
# Training Directory name
TRAINING_DIR = 'cloudmlemodels'

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 
os.environ['TFVERSION'] = '1.13'  # Tensorflow version

In [67]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [68]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud Machine Learning Engine API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Getting the service account email associated with the Cloud Machine Learning Engine API
Authorizing the Cloud ML Service account service-166802608223@cloud-ml.google.com.iam.gserviceaccount.com to access files in data-etl-process1


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   235    0   235    0     0    118      0 --:--:--  0:00:01 --:--:--   118
No changes to gs://data-etl-process1/
No changes to gs://data-etl-process1/all_articles_with_features.pickle
No changes to gs://data-etl-process1/articles_topics.csv
No changes to gs://data-etl-process1/category_id.pickle
No changes to gs://data-etl-process1/feed_data5.pickle
No changes to gs://data-etl-process1/final_views_data.xlsx
No changes to gs://data-etl-process1/Arti_info/april1.xlsx
No changes to gs://data-etl-process1/Arti_info/april3.xlsx
No changes to gs://data-etl-process1/Arti_info/dec1.xlsx
No changes to gs://data-etl-process1/Arti_info/april2.xlsx
No changes to gs://data-etl-process1/Arti_info/dec3.xlsx
No changes to gs://data-etl-process1/Arti_info/feb1.xlsx
No changes to gs://data-etl-process1/Arti_info/jan1.xlsx
No changes to gs://dat

In [71]:
#%%bash
# This is so that the trained model is started fresh each time. However, this needs to be done before 
# tensorboard is started

#echo $OUTDIR $REGION $JOBNAME
os.environ["PROJECT"]



'tatvic-gcp-dev-team'

In [ ]:
%%bash
# Setup python so it sees the task module which controls the model.py
#RUNNING locally
export PYTHONPATH=${PYTHONPATH}:${PWD}/${MODEL_NAME}

python -m trainer.task

In [73]:
%%bash
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
echo $PYTHONPATH

:/home/psdhruv/virality/trainer


In [81]:
%%bash
#running locally using ai platform
OUTDIR= ${PWD}/cloudmlemodels

bash: line 2: /home/psdhruv/virality/cloudmlemodels: Is a directory


CalledProcessError: Command 'b'#running locally using ai platform\nOUTDIR= ${PWD}/cloudmlemodels\n'' returned non-zero exit status 126

In [82]:
%%bash
#running locally using ai platform
#OUTDIR= ${PWD}/cloudmlemodels
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=${PWD}/trainer \
   -- \
   --train_dir=${PWD}/final_data/df_train.csv \
   --valid_dir=${PWD}/final_data/df_valid.csv  \
   --job-dir=${PWD}/cloudmlemodels \
   --OUTDIR=${PWD}/cloudmlemodels

W0707 17:07:31.335862 139647895070464 deprecation_wrapper.py:119] From /home/psdhruv/virality/trainer/model.py:10: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0707 17:07:31.336189 139647895070464 deprecation_wrapper.py:119] From /home/psdhruv/virality/trainer/model.py:10: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

I0707 17:07:31.338058 139647895070464 run_config.py:528] TF_CONFIG environment variable: {'job': {'args': ['--train_dir=/home/psdhruv/virality/final_data/df_train.csv', '--valid_dir=/home/psdhruv/virality/final_data/df_valid.csv', '--job-dir=/home/psdhruv/virality/cloudmlemodels', '--OUTDIR=/home/psdhruv/virality/cloudmlemodels'], 'job_name': 'trainer.task'}, 'task': {}, 'environment': 'cloud', 'cluster': {}}
W0707 17:07:31.338421 139647895070464 deprecation_wrapper.py:119] From /home/psdhruv/virality/trainer/model.py:127: The name tf.summary.FileWriterCache is deprecated

In [102]:
%%bash

OUTDIR=gs://${BUCKET}/final_model
echo $OUTDIR
#--staging-bucket=gs://$BUCKET \

gs://data-etl-process1/final_model


In [195]:
%%bash
#hptuning on cloudml
#make sure all data is in buckets.
OUTDIR=gs://${BUCKET}/${TRAINING_DIR}
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
gcloud ai-platform jobs submit training $JOBNAME \
   --config=hyperparam.yaml \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   --job-dir=$OUTDIR \
   --runtime-version=1.13 \
   --python-version 3.5 \
   --scale-tier=STANDARD_1 \
   --region=$REGION \
   --\
   --output_dir=$OUTDIR \

jobId: virality_190711_123025
state: QUEUED


Job [virality_190711_123025] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe virality_190711_123025

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs virality_190711_123025


In [196]:
%%bash
#Submitting job without hptuning
OUTDIR=gs://${BUCKET}/final_model2
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
export PYTHONPATH=${PYTHONPATH}:${PWD}/trainer
gcloud ai-platform jobs submit training $JOBNAME \
   --module-name=trainer.task \
   --package-path=$(pwd)/trainer \
   --job-dir=$OUTDIR \
   --runtime-version=1.13 \
   --python-version 3.5 \
   --scale-tier=STANDARD_1 \
   --region=$REGION \
   --\
   --output_dir=$OUTDIR \

jobId: virality_190711_123440
state: QUEUED


Job [virality_190711_123440] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe virality_190711_123440

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs virality_190711_123440


In [175]:
import pandas as pd
df_train=pd.read_csv("./final_data2/df_valid.csv")

In [205]:
%%bash
gsutil ls gs://${BUCKET}/my_local_model/cloudmlemodels/export/exporter | tail -1

gs://data-etl-process1/my_local_model/cloudmlemodels/export/exporter/1562843199/


In [197]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/final_model2/export/exporter | tail -1)
echo $MODEL_LOCATION

gs://data-etl-process1/final_model2/export/exporter/1562849115/


In [198]:
%%bash
echo $REGION

asia-east1


In [206]:
%%bash
MODEL_NAME="virality_punjabkesri"
MODEL_VERSION="v4"#enter new version
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/my_local_model/cloudmlemodels/export/exporter | tail -1)
#echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ai-platform models create ${MODEL_NAME} --regions="asia-northeast1"
gcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} \
--origin ${MODEL_LOCATION} --runtime-version $TFVERSION --python-version=3.5


Creating version (this might take a few minutes)......
....................................................................................................................................................................................................................................................................done.


In [3]:
#getting predictions from deployed model
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "virality_punjabkesri"
MODEL_VERSION = 'v4'
PROJECT = "tatvic-gcp-dev-team" 
token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
     {'0': 0.0035658744,
 '1': 0.004414475,
 '10': 0.0018055257,
 '11': 0.0014781109,
 '12': 0.0025544928,
 '13': 0.004745611,
 '14': 0.2856712,
 '2': 0.0020961564,
 '3': 0.0041580633,
 '4': 0.5815388,
 '5': 0.0034491895,
 '6': 0.002489877,
 '7': 0.0061961785,
 '8': 0.0018793311,
 '9': 0.09395714,
 'cat': 'sports',
 'hour': 0,
 'length': 302,
 'msaav10': 370.6,
 'num_of_identities': 4,
 'num_of_sentences': 3,
 'other_entities': 0,
 'polarity': 0.10714285714285714,
 'subjectivity': 0.5714285714285714,
 'weekday': 1,
 "upv3hr":55
     }]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"predictions": [286.08355712890625]}]}'


In [333]:
print(response.content)

b'{"predictions": [{"predictions": [147.20956420898438]}]}'


In [16]:
#running locally
%%bash

python -m trainer.task --OUTDIR="./cloudmlemodels"

Process is terminated.
